### Projekt objektově orientovaného programování

---

Finální projekt kurzu Engeto, úvod do OOP.

<br>

**Zadání**

---

Účelem tohoto projektu je vytvořit produkt, který sbírá nabídky nemovitostí z portálu *bezrealitky*. Dále můžeš projekt rozšířit o tyto doplňky. Hotový projekt by měl mít strukturu klasického Pythoního balíčku:

1. Validace & rozšíření dat,
2. uložení dat do DB,
3. vytvořit jednoduchý web s výpisem získaných dat.

<br>

* Projekt **NENÍ** povinný. Každý, kdo má zájem, si jej může vyzkoušet.

<br>

Pokud budeš chtít zpětnou vazbou (konzultaci s lektorem), odevzdej, prosím, tvůj projekt jako repozitář přes *github* a na Slack pošli lektorovi url tohoto repozitáře.

<br>

Projekt bude mít tyto části:
1. [Sběr dat (OOP,  **povinné**),](#Sběr-dat,-OOP)
    - [úvod k vlastnímu projektu,](#Úvod-k-vlastnímu-projektu)
    - [knihovny & výstup,](#Importování-knihoven-&-výstupní-objekt)
    - [vytvoření nového objektu,](#Vytvoření-objektu-pro-naše-data)
    - [parsování vrácených dat,](#Parsování-vrácených-dat)
    - [nové výstupní objekty,](#Vytvoření-pole-nových-objektů-typu-'BRRealEstateOffer')
    - [úkol A,](#Úkol-A)


2. [Analýza dat (pandas, volitelné),](#Analýza-data,-pandas)
    - [úvod,](#Úvod-k-frameworku)
    - [orientace,](#Zobrazení-údajů)
    - [selekce a řazení,](#Výběr-sloupce-&-seřazení)
    - [přidat & odstranit sloupec,](#Nové-sloupečky)
    - [úkol B,](#Úkol-B)


3. [Uložení dat (sqlite3, volitelné),](#Uložení-dat,-sqlite3,)
    - [úvod,](#Úvod-do-SQL)
    - [zápis testovacích hodnot,](#Zápis-do-testovací-databáze)
    - [zápis testovacích hodnot, sqlite3,](#Zápis-pomocí-sqlite)
    - [čtení testovacích hodnot,](#Čtení-hodnot-v-existující-DB)
    - [zápis skutečných hodnot,](#Zápis-hodnot-z-DataFramu-do-nové-tabulky)
    - [úkol C,](#Úkol-C)


4. [Interpretace dat (flask, volitelné ),](#Intepretace-pomocí-webového-frameworku)
    - [úvod,](#Úvod-do-webového-frameworku)
    - [views,hlavní stránka](#Hlavní-stránka)
    - [views,stránka s výsledky,](#Stránka-s-výsledky)
    - [úkol D](#Úkol-D)

<br>

### Sběr dat, OOP

---

###### Úvod k vlastnímu projektu

---

Nachystání **virtuálního prostředí**:
```
$ python -m venv projekt04       # vytvořím virt. prostředí
$ source projekt04/bin/activate  # aktivuji binárku virt. prostředí
```

<br>

Instalace souvisejících **knihoven**:
```
$ pip --version  # kontrola manažeru
$ pip install -r requirements.txt

Collecting beautifulsoup4==4.10.0
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
Collecting certifi==2021.10.8
...
```

<br>

V rootovi projektu vytvoř soubor s dokumentací `README.md`.

<br>

Ten by měl obsahovat aspoň bodově **strukturu** tvého projektu, příklad objektů, se kterými bys chtěl pracovat a jednotlivé komponenty tvého projektu.

<br>

Vytvoř adresář pro nový balíček:
```
/<projekt04>
    ├─requirements.txt
    ├─README.md
    └─<jmeno_balicku>
       ├─__init__.py
       ├─test_db.db
       ├─prod_db.db
       ├─collector.py
       ├─processor.py
       ├─db.py
       ├─flask_app.py
       ├─tests/
       ├─static/
       └─templates/
```

###### Importování knihoven & výstupní objekt

---

In [ ]:
"""
# náhled na údaj
# --------------
example: Dict[str, Any] = 
{'gps': '{"lat":49.23727,"lng":16.58296}',
 'price': 6499000,
 'currency': 'CZK',
 'key_offer_type': 'prodej',
 'key_estate_type': 'byt',
 'key_disposition': '2-1',
 'surface': 60,
 'surface_land': 0}
"""

import json
from urllib.parse import urljoin
from geopy.geocoders import Nominatim
from typing import Dict, List, Union, Any

import requests

<br>

###### Vytvoření objektu pro naše data

---

In [ ]:
class BRRealEstateOffer:
    """Create a new object from the given attributes."""
    offer_count: int = 0
    
    def __init__(self, details: Dict[str, Any]):
        self.id_ = details.get("id")
        self.url = details.get("url")
        self.gps = details.get("gps")
        self.price = details.get("price")
        self.surface = details.get("surface")
        self.currency = details.get("currency")
        self.surface_land = details.get("surfaceLand")
        self.key_offer_type = details.get("keyOfferType")
        self.key_estate_type = details.get("keyEstateType")
        self.key_disposition = details.get("keyDisposition")

    @classmethod
    def add_offer(cls):
        cls.offer_count += 1

    def __repr__(self) -> str:
        return str(f"{self.url}")
    
    # ...
    def full_description(self) -> str:
        return f"{self.key_offer_type}; {self.key_estate_type}; {self.key_disposition}"

###### Vzorový údaj po zapracování

---

In [ ]:
example: Dict[str, Any] = {
    'id': 695305,
    'surface': 60,
    'price': 6499000,
    'currency': 'CZK',
    'surface_land': 0,
    'keyEstateType': 'byt',
    'keyDisposition': '2-1',
    'keyOfferType': 'prodej',
    'gps': '{"lat":49.23727,"lng":16.58296}',
    'url': "https://www.bezrealitky.cz/nemovitosti-byty-domy/695305-nabidka-prodej-bytu-ostruzinova-brno",
}

In [ ]:
offer_1 = BRRealEstateOffer(example)

In [ ]:
offer_1

###### Požadavek na API

---

In [ ]:
class ScraperInitiator:
    """Initiate a new object for the data transfer."""
    
    def __init__(self, url: str, params: Dict[str, str]):
        self.url = url
        self.params = params
        
    def send_post_request(self) -> requests.models.Response:
        return requests.post(self.url, params=self.params)
    
    @staticmethod
    def load_json(response: requests.models.Response) -> List[Dict[str, str]]:
        """Load the 'json' package and read the content from string."""
        return json.loads(response.text)

In [ ]:
session_1 = ScraperInitiator(
    "https://www.bezrealitky.cz/api/record/markers",
    {
        'offerType': 'prodej',
        'submit': '1',
        'boundary': '[[[{"lat":52,"lng":12},{"lat":52,"lng":16},{"lat":50,"lng":16},{"lat":50,"lng":12},{"lat":52,"lng":12}]]]'
    }
)

In [ ]:
json_: List[Dict[str, Any]] = session_1.load_json(
    session_1.send_post_request()
)

In [ ]:
type(json_)

In [ ]:
json_[10]

<br>

###### Parsování vrácených dat

---

In [ ]:
class DataParser:
    """Parse the given data and create cleaner, non-nested python object."""
    
    def __init__(self, data: List[Dict[str, Any]]):
        self.data = data
        self.url: str = "https://www.bezrealitky.cz/nemovitosti-byty-domy/"
        
    def iterate_through_data(self):
        results: list = []
        
        for offer in self.data:
            uri, details = self.parse_main_dict(offer)
            results.append(
                self.extend_dict(url=urljoin(self.url, uri), details=details)
            )
        
        return results

    @staticmethod
    def parse_main_dict(offer: Dict[str, Any]) -> tuple:
        """Parse and return attributes uri, advertEstateOffer."""
        return offer.get("uri"), offer.get("advertEstateOffer")[0]
    
    @staticmethod
    def extend_dict(**kwargs):
        """Create and update a new dictionary object with the given attrs."""
        attributes: Dict[str, str] = {}

        for key, val in kwargs.items():
            if key == "url":
                attributes[key] = val 
            else:
                attributes.update(val)
                
        return attributes

In [ ]:
parser_1 = DataParser(json_)

In [ ]:
testing_list = parser_1.iterate_through_data()

In [ ]:
testing_list[1]

###### Vytvoření pole nových objektů typu 'BRRealEstateOffer'

---

In [ ]:
class BRRealEstateOfferProcessor:
    """Process the given attributes and a new object 'BRRealEstateOffer'."""
    
    def __init__(self, parsed_data: List[Dict[str, Union[str, int]]]):
        self.offers: list = []
        self.parsed_data = parsed_data

    def add_offer(self, dict_data: Dict[str, Union[str, int]]) -> None:
        self.offers.append(
            BRRealEstateOffer(dict_data)
        )

    def process_offers(self) -> None:
        for offer in self.parsed_data:
            self.add_offer(offer)

In [ ]:
proc_1 = BRRealEstateOfferProcessor(testing_list)
proc_1.process_offers()

In [ ]:
proc_1.offers[11].full_description()

###### Úkol A

---

Zapiš si třídy a jejich instance z předchozí sekce do jednoho spustitelného modulu.

<br>

Příklad spuštění v příkazovém řádku:
```
>>> from bezrealitky.collector import run_collector
>>> offers = run_collector()
>>> offers[0]
https://www.bezrealitky.cz/nemovitosti-byty-domy/695659-nabidka-prodej-garaze-stechovicka-hlavni-mesto-praha
```

<br>

### Analýza data, pandas

---

<br>

###### Úvod k frameworku

---
Obecný popisek [zde](https://pypi.org/project/pandas/)

<br>

Nahrátí knihovny:

In [ ]:
import pandas
# import pandas as pd
# from pandas import DataFrame

<br>

Vytvoření `DataFrame`:

In [ ]:
br_dataframe: pandas.core.frame.DataFrame = pandas.DataFrame(testing_list)

<br>

###### Zobrazení údajů

---

In [ ]:
br_dataframe.head()      # 5 prvních údajů

In [ ]:
br_dataframe.describe()  # základní statistika

<br>

###### Výběr sloupce & seřazení

---

In [ ]:
br_dataframe.loc[:6, ["price", "currency", "surface"]]  # selekce řádků a sloupců (list)

In [ ]:
br_dataframe.iloc[:10, 1:5]                             # selekce řádků a sloupců (rozsah)

In [ ]:
br_dataframe.rename(columns={"surface": "surface[m2]"}, inplace=True)

In [ ]:
br_dataframe.sort_values(by=["price"], ascending=False)  # seřaď podle nejvyšší ceny

In [ ]:
pandas.set_option('display.max_colwidth', 150)           # nastav šířku buněk

###### Nové sloupečky

---

In [ ]:
br_dataframe["type_upper"] = [offer.upper() for offer in br_dataframe["keyEstateType"]]

In [ ]:
br_dataframe.head()

In [ ]:
br_dataframe.drop("type_upper", axis=1, inplace=True)

In [ ]:
br_dataframe.head()

<br>

###### Naformátuj mi cenu na čitelnější hodnotu a ulož do nového sloupce

---

In [ ]:
br_dataframe["formatted_price"] = [f"{price:.2f} mil." for price in br_dataframe['price'] / 1000000]

In [ ]:
br_dataframe.head()


###### Vypiš všechny typy nabídek nemovitostí (unikátní hodnoty) a jejich výskyt

---

In [ ]:
br_dataframe["keyEstateType"].unique()

In [ ]:
for estate_type in br_dataframe["keyEstateType"].unique():
    print(f"TYPE: {estate_type:^16}; \
COUNT: {br_dataframe['keyEstateType'].eq(estate_type).sum()}"
    )

In [ ]:
br_dataframe['keyEstateType'].eq('kancelar').sum()

In [ ]:
estate_type = pandas.Series(br_dataframe["keyEstateType"])

In [ ]:
estate_type

In [ ]:
estate_type.unique()


###### Vypiš mi nejdražší a nejlevnější nemovitost

---

<br>

Přilož detail z odkazem, ať můžeš nabídku zkontrolovat:

In [ ]:
br_dataframe["price"].max()                             # max. hodnota
# br_dataframe[br_dataframe["price"] == 0].url          # řádky s hodnotou = 0
# br_dataframe[br_dataframe["price"] > 40_000_000].url  # řádky s hodnotou > 40 M


###### Vytvoř sloupec, který bude obsahovat skutečnou adresu (podle GPS)

---

<br>

Doplň sloupec do DataFramu, který bude obsahovat celou dostupnou adresu:

In [ ]:
br_dataframe.head()

In [ ]:
def replace_gps(gps: str) -> str:
    """Find the address with the gps coordinates."""
    lat, lng = json.loads(gps).values()
    join_coords = ", ".join((str(lat), str(lng)))  # "50.25, 14.51"
    geolocator = Nominatim(
        user_agent="gps_convertor"
    )
        
    return geolocator.reverse(join_coords).address

address = replace_gps('{"lat":50.25203579999999,"lng":14.5187024}')

In [ ]:
address

###### Úkol B

---
1. Zjistit typy nabízených nemovitostí & jejich výskyt, nejvyšší & nejnižší ceny,

<br>

2. Zkus přidat nový sloupeček `formatted_price`, který bude obsahovat naformátovanou cenu<br>
    ```int64: 2980000 -> object: 2.98```

<br>

3. Zkus přidat nový sloupeček `full_address`, kde předáš DataFramu skutečnou adresu, na základě GPS.

<br>

### Uložení dat, sqlite3,

---

<br>

###### Úvod do SQL

---

Oficiální dokumentace pro SQLite [zde](https://sqlite.org/docs.html).

<br>

**Ano**:
- jednoduché, inituitivní,
- zdarma (většinou),
- *serverless* řešení,
- `null`, `integer`, `text`, `real`,

**Ne**:
- neefektivní práce s většími daty (1TB+ ukládat na centralizované db),
- umožňuje, aby v daném okamžiku proběhla pouze jedna operace zápisu ,

<br>



###### Zápis do testovací databáze

---

In [ ]:
!ls -l

In [ ]:
import sqlite3

In [ ]:
# zajistím spojení s novým objektem db
con = sqlite3.connect('testing_db.db')

In [ ]:
# vytvořím objekt kurzoru, skrze který provádím jednotlivé dotazy
cur = con.cursor()

In [ ]:
# vytvořím novou testovací tabulku, v případě že žádná neexistuje
cur.execute('''
CREATE TABLE IF NOT EXISTS example (
id INTEGER PRIMARY KEY,
name TEXT,
email TEXT)
''')

In [ ]:
# Vložím tři různé řádky
cur.execute("INSERT INTO example VALUES (NULL,'Matous','matous@matous.cz')")
cur.execute("INSERT INTO example VALUES (NULL,'Petr','petr@metr.cz')")
cur.execute("INSERT INTO example VALUES (NULL,'Lukas','luki@puky.cz')")

# commitnu změny do tabulky
con.commit()

# ukončím spojení
con.close()

In [ ]:
!ls -l

###### Zápis pomocí sqlite

---

In [ ]:
# zajistím spojení s novým objektem db
con = sqlite3.connect('testing_db.db')
cur = con.cursor()

sl_ = {
    "name": "Marek",
    "email": "marek.parek@email.cz"
}

cur.execute(f"""INSERT INTO example ({','.join(sl_.keys())}) \
            VALUES ({','.join(['?'] * len(sl_))})""", tuple(sl_.values()))
# 'INSERT INTO example (name,email) VALUES (?,?)', ("Marek","marek.parek@email.cz")

con.commit()

# ukončím spojení
con.close()

###### Čtení hodnot v existující DB

---

In [ ]:
con = sqlite3.connect('testing_db.db')
cur = con.cursor()

# označím všechny sloupce v  tabulce 'testing_db.sql'
cur.execute("SELECT * FROM example")

# nainicializuji je do proměnné 'data'
data = cur.fetchall()

# vypisuji řádek po řádku pomocí iterování skrze objekt
for row in data:
    print(row)

con.close()

###### Context manager

---

In [ ]:
with sqlite3.connect('testing_db.db') as con:
    cur = con.cursor()
    
    cur.execute("SELECT * FROM example")
    data = cur.fetchall()

    for row in data:
        print(row)

###### Zápis hodnot z DataFramu do nové tabulky

---

In [ ]:
with sqlite3.connect('br_offers.db') as con:
    cur = con.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS offers (
    id INTEGER PRIMARY KEY,
    url TEXT,
    gps TEXT,
    price INTEGER,
    currency TEXT,
    keyOfferType TEXT,
    keyEstateType TEXT,
    keyDisposition TEXT,
    surface INTEGER,
    surfaceLand INTEGER,
    offer_id INTEGER,
    formatted_price TEXT,
    full_address TEXT
    )
    """
    )

    con.commit()

    br_dataframe.to_sql(
        'offers', con, if_exists='replace',
        index=True,
    )

###### Úkol C

---

Zkus zapsat získaná data pomocí objektu v Pythonu jako databázový systém.

<br>

### Intepretace pomocí webového frameworku

---

<br>

###### Úvod do webového frameworku

---

Dokumentaci lze najít [zde](https://flask.palletsprojects.com/en/2.0.x/)

<br>



###### Hlavní stránka

---

Výstup:
```
Ahoj, na moji hlavni strance!
```

In [ ]:
# flask_app.py bez .html
from flask import Flask  # naimportuj třídu 'Flask'

app = Flask(__name__)    # vytvoříme instanci se jménem akt. souboru 

@app.route("/")          # pomocí dekorátoru vytvořím cílové url
def main_page():         # funkce pro úvodní stránku
    return "<p>Ahoj, na moji hlavni strance</p>"

# if __name__ == "__main__":
#     app.run()

In [ ]:
# flask_app.py s templates/main_page.html
from flask import Flask, render_template

app = Flask(__name__)


@app.route("/")
def main_page():
    return render_template("main_page.html")


# if __name__ == "__main__":
#     app.run(debug=True)    # po každé úpravě dojde na refresh

###### HTML main_page

---
```html
<!-- main_page.html -->
<html>
  <head>
    <title>Main page</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
  </head>
<body>
    <div class="row-3" id="br_offers">
      <div class="row 4">
        <h1 id="title">Ahoj na mojí hlavní stránce!</h1>
      </div>
    </div>
</body
```

###### Stránka s výsledky

---

In [ ]:
# main_page.py
import os
import sqlite3

from flask import Flask, render_template, request

app = Flask(__name__)


@app.route("/")
def main_page():
    return render_template("main_page.html")


@app.route("/results")
def results():
    with sqlite3.connect("01_testing.db") as con:
        cur = con.cursor()

        cur.execute("SELECT * FROM example")
        data = cur.fetchall()

    return render_template("results.html", result=data)

###### HTML results.html
---

Výstup:
```
(1, 'Matous', 'matous@matous.cz')
(2, 'Petr', 'petr@metr.cz')
(3, 'Lukas', 'luki@puky.cz')
(4, 'Marek', 'marek.parek@email.cz')
```


<br>

```html
<!-- results.html -->
<html>
  <head>
    <title>Results</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
  </head>
<body>
    <div class="container" id="results">

      <div class="row 4">
        <h1 id="title">Výsledky</h1>
      </div>

      <div class="row 3">
        <table id="ner-table" data-test-id="ner-table" class="table table-bordered">
          <thead class="thead-dark">
            <tr>
              <th>id</th>
              <th>jmeno</th>
              <th>email</th>
            </tr>
          </thead>
          <tbody>
            {% for row in result %}
            <tr>
                <td>{{ row }}</td>
            </tr>
            {% endfor %}
          </tbody>
        </table>
      </div>
    </div>
  </body>
</html>
```


###### Doplníme indexy

---

Výstup:
```
1 	Matous 	matous@matous.cz
2 	Petr 	petr@metr.cz
3 	Lukas 	luki@puky.cz
4 	Marek 	marek.parek@email.cz
```

<br>

```html
          <thead class="thead-dark">
            <tr>
              <th>id</th>
              <th>jmeno</th>
              <th>email</th>
            </tr>
          </thead>
          <tbody>
            {% for row in result %}
            <tr>
                <td>{{ row[0] }}</td>
                <td>{{ row[1] }}</td>
                <td>{{ row[2] }}</td>
            </tr>
            {% endfor %}
          </tbody>
```

```html
<html>
  <head>
    <title>Results</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
  </head>
<body>
    <div class="container p-3" id="app">

      <div class="row my-4">
        <h1 id="app-title">Results</h1>
      </div>

      <div class="row my-3">
        <table id="ner-table" data-test-id="ner-table" class="table table-bordered">
          <thead class="thead-dark">
            <tr>
              <th>id</th>
              <th>url</th>
              <th>price</th>
              <th>currency</th>
              <th>type</th>
              <th>surface</th>
            </tr>
          </thead>
          <tbody>
            {% for row in result %}
            <tr>
                <td>{{ row[10] }}</td>
                <td>{{ row[1] }}</td>
                <td>{{ row[3] }}</td>
                <td>{{ row[4] }}</td>
                <td>{{ row[6] }}</td>
                <td>{{ row[8] }}</td>
            </tr>
            {% endfor %}
          </tbody>
        </table>
      </div>
    </div>
  </body>
</html>
```

###### Úkol D

---

Zkus zobrazit získaná data pomocí frameworku *Flask* na localhostu. Rozděl jednotlivé údaje do tabulky a vypiš některé hodnoty.

In [ ]:
class Trida:
    def __init__():
    
    def neco():

---